<a href="https://colab.research.google.com/github/JisungJang-AI-BIO/LAIDD_Developability/blob/main/Boltz-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. 환경 설정: Boltz 및 의존성 패키지 설치

`boltz==2.1.1`의 공식 요구사항(`pytorch-lightning==2.5.0`)과 호환되는 PyTorch 버전(`2.3.0`)을 명시하여 모든 패키지를 한 번에 설치합니다.

In [2]:
# 1단계: 기존 패키지를 모두 제거하여 깨끗한 상태를 보장합니다.
print("에러 발생 시 기존 PyTorch 및 Boltz 관련 패키지를 제거합니다...")
#!pip uninstall torch torchvision torchaudio boltz -y -q

# 모든 제약조건(boltz, torch==2.2.0 등)을 단일 명령어로 전달하여 한 번에 설치합니다.
# --extra-index-url을 사용하여 PyPI와 PyTorch 서버를 모두 검색하도록 합니다.
print("\nBoltz와 호환되는 모든 패키지를 동시에 설치합니다...")
!pip install boltz torch==2.2.0 torchvision==0.17.0 torchaudio==2.2.0 --extra-index-url https://download.pytorch.org/whl/cu121 -q
# 결과물 시각화 패키지
!pip install py3Dmol -q
# 설치된 버전을 최종 확인하여, torch가 의도한 대로 2.2.0으로 설치되었는지 검증합니다.
print("\n설치된 패키지 버전을 확인합니다.")
!pip show torch torchvision

에러 발생 시 기존 PyTorch 및 Boltz 관련 패키지를 제거합니다...

Boltz와 호환되는 모든 패키지를 동시에 설치합니다...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.3/757.3 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

---

## 2. Trastuzumab Fab 단편 구조 예측 (실행 예제)

항원 없이 Trastuzumab 항체의 가변 영역(VH, VL)만으로 Fab 구조를 예측합니다. Colab 환경에서 정상적으로 실행됩니다.

In [4]:
import yaml
import datetime
import os
import glob
import py3Dmol

# 입력 데이터: Trastuzumab Fab (VH, VL)
heavy_chain_sequence = "EVQLVESGGGLVQPGGSLRLSCAASGFNIKDTYIHWVRQAPGKGLEWVARIYPTNGYTRYADSVKGRFTISADTSKNTAYLQMNSLRAEDTAVYYCSRWGGDGFYAMDYWGQGTLVTVSS"
light_chain_sequence = "DIQMTQSPSSLSASVGDRVTITCRASQDVNTAVAWYQQKPGKAPKLLIYSASFLYSGVPSRFSGSRSGTDFTLTISSLQPEDFATYYCQQHYTTPPTFGQGTKVEIK"
output_name = "Trastuzumab_Fab_prediction"

# YAML 파일 생성
input_data = {
    'sequences': [
        {'protein': {'id': 'H', 'sequence': heavy_chain_sequence}},
        {'protein': {'id': 'L', 'sequence': light_chain_sequence}}
    ],
    'name': output_name
}
yaml_path = f"{output_name}.yaml"
with open(yaml_path, 'w') as f:
    yaml.dump(input_data, f, indent=2, sort_keys=False)
print(f"✅ YAML 파일 생성 완료: {yaml_path}")

# Boltz 예측 실행
timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
output_dir = f"boltz_run_{timestamp}"
print(f"✅ 결과 저장 폴더: {output_dir}")
print("\nBoltz-2 예측을 시작합니다...")
!boltz predict {yaml_path} --use_msa_server --output_format mmcif --out_dir {output_dir}
print("\n--- 예측 프로세스 완료 ---")

# 결과 시각화
print("\n결과 시각화를 시작합니다...")
mmcif_files = glob.glob(os.path.join(output_dir, '**', '*.mmcif'), recursive=True)
try:
    if not mmcif_files:
        raise FileNotFoundError(f"'{output_dir}' 폴더에서 .mmcif 파일을 찾을 수 없습니다.")
    mmcif_files.sort()
    mmcif_file_path = mmcif_files[0]
    print(f"✅ 발견된 mmcif 파일: {mmcif_file_path}")
    with open(mmcif_file_path, 'r') as f:
        mmcif_data = f.read()
    view = py3Dmol.view(width=800, height=600)
    view.addModel(mmcif_data, "mmcif")
    view.setStyle({'chain':'H'}, {'cartoon': {'color':'blue'}})
    view.setStyle({'chain':'L'}, {'cartoon': {'color':'red'}})
    view.zoomTo()
    view.show()
except FileNotFoundError as e:
    print(f"오류: {e}")
except Exception as e:
    print(f"시각화 중 오류가 발생했습니다: {e}")

✅ YAML 파일 생성 완료: Trastuzumab_Fab_prediction.yaml
✅ 결과 저장 폴더: boltz_run_20250705_103040

Boltz-2 예측을 시작합니다...
Checking input data.
Running predictions for 1 structure
Processing input data.
  0% 0/1 [00:00<?, ?it/s]Generating MSA for Trastuzumab_Fab_prediction.yaml with 2 protein entities.

  0% 0/300 [00:00<?, ?it/s]
SUBMIT:   0% 0/300 [00:00<?, ?it/s]
COMPLETE:   0% 0/300 [00:00<?, ?it/s]
COMPLETE: 100% 300/300 [00:00<00:00, 372.26it/s] 

  0% 0/300 [00:00<?, ?it/s]
SUBMIT:   0% 0/300 [00:00<?, ?it/s]
COMPLETE:   0% 0/300 [00:00<?, ?it/s]
COMPLETE: 100% 300/300 [00:01<00:00, 220.09it/s] 
100% 1/1 [00:02<00:00,  2.94s/it]
Traceback (most recent call last):
  File "/usr/local/bin/boltz", line 8, in <module>
    sys.exit(cli())
             ^^^^^
  File "/usr/local/lib/python3.11/dist-packages/click/core.py", line 1157, in __call__
    return self.main(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/click/core.py", line 1078, in main

---

## 3. (참고) Trastuzumab - HER2 항원 결합 구조 예측

⚠️ **경고**: 이 예제는 항원의 크기가 매우 커서, 일반 Colab GPU 환경에서는 **메모리 부족(Out of Memory)으로 실패할 가능성이 매우 높습니다.** 더 높은 사양의 GPU 환경에서 테스트하거나 참고용으로만 활용하는 것을 권장합니다.

In [ ]:
import yaml
import datetime
import os
import glob
import py3Dmol

# 입력 데이터: Trastuzumab(VH, VL)과 HER2 항원
heavy_chain_sequence_orig = "EVQLVESGGGLVQPGGSLRLSCAASGFNIKDTYIHWVRQAPGKGLEWVARIYPTNGYTRYADSVKGRFTISADTSKNTAYLQMNSLRAEDTAVYYCSRWGGDGFYAMDYWGQGTLVTVSS"
light_chain_sequence_orig = "DIQMTQSPSSLSASVGDRVTITCRASQDVNTAVAWYQQKPGKAPKLLIYSASFLYSGVPSRFSGSRSGTDFTLTISSLQPEDFATYYCQQHYTTPPTFGQGTKVEIK"
antigen_sequence_orig = "TQVCTGTDMKLRLPASPETHLDMLRHLYQGCQVVQGNLELTYLPTNASLSFLQDIQEVQGYVLIAHNQVRQVPLQRLRIVRGTQLFEDNYALAVLDNGDPLNNTTPVTGASPGGLRELQLRSLTEILKGGVLIQRNPQLCYQDTILWKDIFHKNNQLALTLIDTNRSRACHPCSPMCKGSRCWGESSEDCQSLTRTVCAGGCARCKGPLPTDCCHEQCAAGCTGPKHSDCLACLHFNHSGICELHCPALVTYNTDTFESMPNPEGRYTFGASCVTACPYNYLSTDVGSCTLVCPLHNQEVTAEDGTQRCEKCSKPCARVCYGLGMEHLREVRAVTSANIQEFAGCKKIFGSLAFLPESFDGDPASNTAPLQPEQLQVFETLEEITGYLYISAWPDSLPDLSVFQNLQVIRGRILHNGAYSLTLQGLGISWLGLRSLRELGSGLALIHHNTHLCFVHTVPWDQLFRNPHQALLHTANRPEDECVGEGLACHQLCARGHCWGPGPTQCVNCSQFLRGQECVEECRVLQGLPREYVNARHCLPCHPECQPQNGSVTCFGPEADQCVACAHYKDPPFCVARCPSGVKPDLSYMPIWKFPDEEGACQPCPINCTHSCVDLDDKGCPAEQRASPLT"
output_name_orig = "Trastuzumab_HER2_binding_prediction"

# YAML 파일 생성
input_data_orig = {
    'sequences': [
        {'protein': {'id': 'H', 'sequence': heavy_chain_sequence_orig}},
        {'protein': {'id': 'L', 'sequence': light_chain_sequence_orig}},
        {'protein': {'id': 'A', 'sequence': antigen_sequence_orig}}
    ],
    'name': output_name_orig
}
yaml_path_orig = f"{output_name_orig}.yaml"
with open(yaml_path_orig, 'w') as f:
    yaml.dump(input_data_orig, f, indent=2, sort_keys=False)
print(f"✅ YAML 파일 생성 완료: {yaml_path_orig}")

# Boltz 예측 실행
timestamp_orig = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
output_dir_orig = f"boltz_run_{timestamp_orig}"
print(f"✅ 결과 저장 폴더: {output_dir_orig}")
print("\nBoltz-2 예측을 시작합니다...")
!boltz predict {yaml_path_orig} --use_msa_server --output_format mmcif --out_dir {output_dir_orig}
print("\n--- 예측 프로세스 완료 ---")

# 결과 시각화
print("\n결과 시각화를 시작합니다...")
mmcif_files_orig = glob.glob(os.path.join(output_dir_orig, '**', '*.mmcif'), recursive=True)
try:
    if not mmcif_files_orig:
        raise FileNotFoundError(f"'{output_dir_orig}' 폴더에서 .mmcif 파일을 찾을 수 없습니다.")
    mmcif_files_orig.sort()
    mmcif_file_path_orig = mmcif_files_orig[0]
    print(f"✅ 발견된 mmcif 파일: {mmcif_file_path_orig}")
    with open(mmcif_file_path_orig, 'r') as f:
        mmcif_data_orig = f.read()
    view = py3Dmol.view(width=800, height=600)
    view.addModel(mmcif_data_orig, "mmcif")
    view.setStyle({'chain':'H'}, {'cartoon': {'color':'blue'}})
    view.setStyle({'chain':'L'}, {'cartoon': {'color':'green'}})
    view.setStyle({'chain':'A'}, {'cartoon': {'color':'red'}})
    view.zoomTo()
    view.show()
except FileNotFoundError as e:
    print(f"오류: {e}")
except Exception as e:
    print(f"시각화 중 오류가 발생했습니다: {e}")